# RGR Stock Price Forecasting Project - Part 5

Author: Jack Wang

---

## Problem Statement

Stock prices are hard to predict because they are not only affected by the performance of the underlying companies but also the expectations from the general public. As known, the stock price of firearm companies are highly correlated to the public opinions toward gun control. My model intends to predict the stock price of one of the largest firearm company in the states, RGR (Sturm, Ruger & Co., firearm company), by using its historical stock price, public opinions toward gun control, and its financial reports to SEC. 

## Executive Summary

The goal of my projcet is to build a **time series regression model** that predicts the stock price of RGR. The data I am using would be historical stock price from [Yahoo Finance](https://finance.yahoo.com/quote/RGR/history?p=RGR), twitter posts scraped from [twitter](https://twitter.com/), subreddit posts mentioned about gun control, and also the financial reports to [SEC](https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000095029&type=&dateb=&owner=exclude&count=100). I will do sentiment analysis on the text data and time series modeling on the historical stock price data. The model will be evaluated using MSE.

## Content

This project consists of 7 Jupyter notebooks:
- Part-1-stock-price-data
- Part-2-twitter-scraper
- Part-3-twitter-data-cleaning
- Part-4-reddit-data-scraper
- ***Part-5-reddit-data-cleaning***
    - [Politics Subreddit](#Politics-Subreddit)
    - [Guns Subreddit](#Guns-Subreddit)
    - [Combining Both Subreddit](#Combining-Both-Subreddit)
- Part-6-sec-data-cleaning
- Part-7-modeling-and-evaluation


---


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import re

from datetime import datetime
from nltk.tokenize import RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# Historical subreddit posts 
df_guns = pd.read_csv("../data/reddit/reddit_guns_2016_to_2019.csv")
df_politics = pd.read_csv("../data/reddit/reddit_politics_2016_to_2019.csv")

### Politics Subreddit

In [3]:
df_politics.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,3yz53c,Obama to reverse Bill Clinton gun control stra...,http://www.washingtontimes.com/news/2015/dec/3...,drewiepoodle,38,2015-12-31 17:40:22,6,/r/politics/comments/3yz53c/obama_to_reverse_b...,NaN
1,3z0ga7,"[Poll] Americans, what is your stance on gun c...",http://strawpoll.me/6419380,[deleted],1,2016-01-01 04:06:01,1,/r/politics/comments/3z0ga7/poll_americans_wha...,No Petitions or Polls
2,3z0gni,Obama to impose new gun control curbs next week,https://www.washingtonpost.com/news/post-polit...,[deleted],2,2016-01-01 04:11:22,6,/r/politics/comments/3z0gni/obama_to_impose_ne...,NaN
3,3z0ocw,"Advancing gun control, closing the U.S. milita...",http://www.reuters.com/article/us-usa-obama-id...,TwoGee,1,2016-01-01 06:01:18,1,/r/politics/comments/3z0ocw/advancing_gun_cont...,NaN
4,3z11q9,The List [Gun Control],http://www.idiotfreezone.com/Politics/what-do-...,StonerMeditation,0,2016-01-01 08:24:14,39,/r/politics/comments/3z11q9/the_list_gun_control/,Unacceptable Domain


In [4]:
# drop unnecessary columns
df_politics = df_politics.drop(columns=['Flair', 'Url', 'Author', 'Permalink', 'Post ID'])

# covert time to date
df_politics['Publish Date'] = pd.to_datetime(df_politics['Publish Date']).dt.date

# drop extra row
df_politics = df_politics[1:].copy()

In [5]:
df_politics.head()

,Title,Score,Publish Date,Total No. of Comments
1,"[Poll] Americans, what is your stance on gun c...",1,2016-01-01,1
2,Obama to impose new gun control curbs next week,2,2016-01-01,6
3,"Advancing gun control, closing the U.S. milita...",1,2016-01-01,1
4,The List [Gun Control],0,2016-01-01,39
5,Obama to impose new gun control curbs next week,2,2016-01-01,128


**Clean up & add features**

In [6]:
# remove any http urls in title
df_politics['Title'] = [re.sub(r'http\S+', '', post).strip() for post in df_politics['Title']]

# instatiate the tokenizer
tknr = RegexpTokenizer(r'[a-zA-Z&0-9]+')

# start with empty lists
tokens = []

# fill the list with tokenized versions of each post title
for post in df_politics['Title']:
    tokens.append(" ".join(tknr.tokenize(post.lower())))
df_politics['Title'] = tokens

# compound score added
sia = SentimentIntensityAnalyzer()

# create function to return compound score
def get_compound(text):
    return sia.polarity_scores(text)['compound']

# add compound score features for title and tac column
df_politics['title_compound'] = df_politics['Title'].map(lambda x : get_compound(x))

# include sum values by date
df_p1 = df_politics.groupby(['Publish Date']).sum()

# include count values by date
df_p2 = df_politics.groupby(['Publish Date']).count()

# include mean values by datee
df_p3 = df_politics.groupby(['Publish Date']).mean()

# merge all dataframe
df_pol_final = pd.merge(df_p3,pd.merge(df_p1, df_p2,left_index = True, right_index = True), left_index = True, right_index=True)

# drop extra columns
df_pol_final = df_pol_final.drop(columns = ['title_compound_x', 'Score_y', 'title_compound_y', 'Total No. of Comments_y'])

# rename column names
df_pol_final = df_pol_final.rename(columns={'Score': 'redd_pol_score_mean',
                   'Total No. of Comments': 'redd_pol_comment_mean',
                   'title_compound': 'redd_pol_compound_mean',
                   'Score_x': 'redd_pol_score_sum',
                   'Total No. of Comments_x': 'redd_pol_comment_sum',
                   'Title': 'redd_pol_post_count'
                  })

# keep the date (not just the index)
df_pol_final['date']=df_pol_final.index

In [7]:
# final cleaned dataframe
df_pol_final.head()

,redd_pol_score_mean,redd_pol_comment_mean,redd_pol_compound_mean,redd_pol_score_sum,redd_pol_comment_sum,redd_pol_post_count,date
Publish Date,,,,,,,
2016-01-01,7.555556,38.222222,-0.549633,68,344,9,2016-01-01
2016-01-02,1.625000,3.000000,-0.426113,13,24,8,2016-01-02
2016-01-03,68.200000,111.000000,-0.317500,682,1110,10,2016-01-03
2016-01-04,4.666667,16.166667,-0.352606,84,291,18,2016-01-04
2016-01-05,4.828571,16.228571,-0.348534,169,568,35,2016-01-05


In [8]:
# export to csv file
df_pol_final.to_csv("../data/reddit/final_pol_2016_to_2019.csv", index= False)

### Guns Subreddit

In [9]:
df_guns.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,3z0z4h,Obama to impose new gun control curbs next week,https://www.washingtonpost.com/news/post-polit...,Tooschbag,1,2016-01-01 08:00:52,1,/r/guns/comments/3z0z4h/obama_to_impose_new_gu...,NaN
1,3z55nt,"Obama To Unveil ""Multiple Gun Control"" Executi...",http://www.zerohedge.com/news/2016-01-01/obama...,[deleted],0,2016-01-02 06:07:19,3,/r/guns/comments/3z55nt/obama_to_unveil_multip...,NaN
2,3zaoa8,The Gun Control Debate (for grown ups),http://www.viralcodex.com/2016/01/02/the-gun-c...,Viral_Codex,1,2016-01-03 09:55:38,0,/r/guns/comments/3zaoa8/the_gun_control_debate...,NaN
3,3zl2lc,President Obama's Executive Order on Gun Control,http://bearingarms.com/sound-fury-signifying-n...,[deleted],0,2016-01-05 09:11:21,21,/r/guns/comments/3zl2lc/president_obamas_execu...,NaN
4,3zl9rk,"American gun owners, what's your opinion on Ob...",https://www.reddit.com/r/guns/comments/3zl9rk/...,[deleted],1,2016-01-05 09:54:07,0,/r/guns/comments/3zl9rk/american_gun_owners_wh...,NaN


**Clean up & add features**

In [10]:
# drop unnecessary cloumns
df_guns = df_guns.drop(columns=['Flair', 'Url', 'Author', 'Permalink', 'Post ID'])

# convert time to date
df_guns['Publish Date'] = pd.to_datetime(df_guns['Publish Date']).dt.date

# remove any http urls
df_guns['Title'] = [re.sub(r'http\S+', '', post).strip() for post in df_guns['Title']]

# instatiate the tokenizer
tknr = RegexpTokenizer(r'[a-zA-Z&0-9]+')

# start with empty lists
tokens = []

# fill the list with tokenized versions of each post title
for post in df_guns['Title']:
    tokens.append(" ".join(tknr.tokenize(post.lower())))
df_guns['Title'] = tokens

# compound score added
sia = SentimentIntensityAnalyzer()

# create function to return compound score
def get_compound(text):
    return sia.polarity_scores(text)['compound']

# add compound score features for title and tac column
df_guns['title_compound'] = df_guns['Title'].map(lambda x : get_compound(x))

# add sum column by date
df_g1 = df_guns.groupby(['Publish Date']).sum()

# add count column by date
df_g2 = df_guns.groupby(['Publish Date']).count()

# add mean column by date
df_g3 = df_guns.groupby(['Publish Date']).mean()

# merge dataframes
df_gun_final = pd.merge(df_g3,pd.merge(df_g1, df_g2,left_index = True, right_index = True), left_index = True, right_index=True)

# drop extra columns
df_gun_final = df_gun_final.drop(columns = ['title_compound_x', 'Score_y', 'title_compound_y', 'Total No. of Comments_y'])

# rename columns
df_gun_final = df_gun_final.rename(columns={'Score': 'redd_gun_score_mean',
                   'Total No. of Comments': 'redd_gun_comment_mean',
                   'title_compound': 'redd_gun_compound_mean',
                   'Score_x': 'redd_gun_score_sum',
                   'Total No. of Comments_x': 'redd_gun_comment_sum',
                   'Title': 'redd_gun_post_count'
                  })

# add date to column
df_gun_final['date']=df_gun_final.index

In [11]:
df_gun_final.head()

,redd_gun_score_mean,redd_gun_comment_mean,redd_gun_compound_mean,redd_gun_score_sum,redd_gun_comment_sum,redd_gun_post_count,date
Publish Date,,,,,,,
2016-01-01,1.000,1.000,-0.557400,1,1,1,2016-01-01
2016-01-02,0.000,3.000,-0.340000,0,3,1,2016-01-02
2016-01-03,1.000,0.000,-0.340000,1,0,1,2016-01-03
2016-01-05,0.875,5.875,-0.335113,7,47,8,2016-01-05
2016-01-06,0.000,19.000,-0.200933,0,57,3,2016-01-06


In [12]:
# export to csv file
df_gun_final.to_csv("../data/reddit/final_gun_2016_to_2019.csv", index= False)

### Combining Both Subreddit

In [37]:
df_final = pd.merge(df_gun_final, df_pol_final, how='outer')#left_on='date', right_on='date')

In [38]:
df_pol_final.shape

(725, 7)

In [40]:
df_gun_final.shape

(289, 7)

In [43]:
df_final = df_final.fillna(0)

In [50]:
df_final['index_date']=pd.to_datetime(df_final['date'])
df_final = df_final.set_index(['index_date'])

In [78]:
df_final = df_final['2016-10-01':'2019-10-01']

In [82]:
df_final.to_csv("../data/reddit/reddit.csv", index = False)